# 表单数据
如果接收到的不是json数据，而是表单数据，可以使用 Form 类来声明参数。例如：

需要 pip install python-multipart

使用方法类似于 Body()：不同的是 Body接受json，而 Form 接受表单数据。 **只能二选一，不能并存**

```python
async def login(username: str = Form(), password: str = Form()):
```

In [ ]:
import uvicorn
from fastapi import FastAPI, Form
app = FastAPI()
@app.post("/login/")
async def login(username: str = Form(), password: str = Form()):
    return {"username": username}
if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

"""
data = {"username": "mockuser", "password": "123456"}
res = requests.post(url, data=data, headers=headers)  ！！！！！！！！！！！！这里用的是data参数，而不是json

此时请求体的编码是 application/x-www-form-urlencoded，不是 application/json。
"""

# 请求文件
上传文件以「表单数据」形式发送。

定义文件参数时使用 UploadFile：art。

```python
async def create_upload_file(file: UploadFile): 
```

In [ ]:
import uvicorn
from fastapi import FastAPI, UploadFile
app = FastAPI()

@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile): # 定义文件参数
    return {"filename": file.filename}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

"""
url = "http://127.0.0.1:8009/uploadfile/"
files = {"file": open('19.JPG', "rb")}
res = requests.post(url, files=files)    !!!!!!!!!!!!! 用的是files参数
res.text

返回 {"filename":"19.JPG"}'
"""

UploadFile 的属性如下：

1. filename：上传文件名字符串（str），例如， myimage.jpg； 
2. content_type：内容类型（MIME 类型 / 媒体类型）字符串（str），例如，image/jpeg； 
3. file： SpooledTemporaryFile（ file-like 对象）。其实就是 Python文件，可直接传递给其他预期 file-like 对象的函数或

#### UploadFile 属性与常用方法说明
- **filename**：上传文件名，例如 `myimage.jpg`。
- **content_type**：MIME 媒体类型，例如 `image/jpeg`。
- **file**：内部实际使用 `SpooledTemporaryFile`，可直接传给其他需要 file-like 对象的函数。

+ `write(data)`：把 `str` 或 `bytes` 写入文件。
- `read(size)`：按字节数读取内容；缺省时读完整文件。
- `seek(offset)`：移动文件指针，例如 `await myfile.seek(0)` 回到开头，便于重复读取。
- `close()`：关闭临时文件。

> 由于这些方法本质上是 async 包装，FastAPI 会在后台线程池运行真实的文件操作，需要使用 `await`。
- 在 `async def` 路径函数中读取文件：`contents = await myfile.read()`。
- 在普通 `def` 路径函数中：可直接访问 `myfile.file.read()`（同步文件对象）。

+ **示例**：
```python
@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile):
    contents = await file.read()  # async 读取
    await file.seek(0)            # 若需要再次读取
    return {
        "filename": file.filename,
        "content_type": file.content_type,
        "size": len(contents)
    }
```

#### 可选上传
`async def create_upload_file(file: UploadFile | None = None):`

这样可以上传也可以不上传

import uvicorn
from fastapi import FastAPI, UploadFile
app = FastAPI()

@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile | None = None):
    if not file:
        return {"message": "No upload file sent"}
    else:
        return {"filename": file.filename}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

#### 多文件上传
用List包装
```python
async def create_upload_files(files: list[UploadFile]):
```

In [ ]:
import uvicorn
from fastapi import FastAPI, UploadFile
app = FastAPI()

@app.post("/uploadfiles/")
async def create_upload_files(files: list[UploadFile]):
    return {"filenames": [file.filename for file in files]}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

"""
url = "http://127.0.0.1:8009/uploadfiles/"
files = [("files", open('19.JPG', "rb")),("files", open('89.JPG', "rb"))]
res = requests.post(url, files=files)
res.text

输出：'{"filenames":["19.JPG","89.JPG"]}'
"""

# 带前端的示例

In [ ]:
import uvicorn
from fastapi import FastAPI, UploadFile
from fastapi.responses import HTMLResponse
app = FastAPI()

@app.post("/uploadfiles/")
async def create_upload_files(files: list[UploadFile]):
    return {"filenames": [file.filename for file in files]}

@app.get("/")
async def main():
    content = """
<body>
<form action="/uploadfiles/" enctype="multipart/form-data" method="post">
<input name="files" type="file" multiple>
<input type="submit">
</form>
</body>
    """
    return HTMLResponse(content=content)

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [11884]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8009 (Press CTRL+C to quit)
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:58364 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [11884]
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [11884]


: 